In [34]:
import pandas as pd

In [44]:
# Retrieve the calculated DNA time (very high risk, surgeon clinics, new patients only)
%store -r dna_very_high_surgeon_new

In [35]:
# Import future ENT appointments
data = pd.read_csv("../future_appts_trans_ENT.csv")
data

,appointment_id,age_deid,bmi_deid,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,substance_misuse,...,appt_date,ethnicity_trans,tfc_name_trans,patient_on_multiple_pathways_trans,appt_month_trans,hospital_service_display_trans,service_mapping_trans,reason_display_trans,appointment_type_trans,did_not_attend_risk_group_trans
0,17479744,57,27.45,1,0,1,1,0,0,0,...,2024-06-26,3,7,0,6,0,10,1,0,3
1,17631073,17,27.45,0,1,0,2,0,0,0,...,2024-05-07,3,7,0,5,0,10,1,0,3
2,17659262,27,21.70,0,0,0,4,0,0,0,...,2024-05-07,5,7,0,5,0,10,2,0,2
3,17659348,57,21.70,0,0,0,3,0,0,0,...,2024-05-07,5,7,0,5,0,10,2,0,3
4,17659361,27,27.45,0,1,0,2,0,0,0,...,2024-05-07,3,7,0,5,0,10,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560,20183628,47,27.45,0,0,1,2,0,0,0,...,2024-06-19,5,7,0,6,0,10,1,0,1
2561,20183649,57,40.00,0,1,0,1,0,0,0,...,2024-06-08,3,7,0,5,0,10,2,0,3
2562,20183782,32,21.70,0,0,0,4,0,0,0,...,2024-05-25,3,7,0,7,0,10,6,0,1
2563,20183795,62,39.45,0,0,0,1,0,0,0,...,2024-06-17,3,7,0,6,0,10,3,0,1


In [40]:
# Filter for surgeons & new appointments only
surgeon_new = data[data["reason_display"].isin(["Adult ENT New", "Adult ENT Thyroid New"])]

In [19]:
# # Group weekswaiting as per NHS guidelines
# data["weekswaiting"]
# data["weekswaiting_grouped"] = pd.cut(data["weekswaiting"], bins=[0, 17, 51, 76, float('inf')], labels=["0-17", "18-51", "52-77", ">78"])
# data[["weekswaiting","weekswaiting_grouped"]]

,weekswaiting,weekswaiting_grouped
0,53.0,52-77
1,82.0,>78
2,55.0,52-77
3,55.0,52-77
4,54.0,52-77
...,...,...
2560,44.0,18-51
2561,13.0,0-17
2562,49.0,18-51
2563,11.0,0-17


### V1: overbook any into very high risk based on weeks waiting

In [45]:
# Sort the appointments by weekswaiting (descending; all except the very high risk ones)
weekswaiting_desc = surgeon_new[surgeon_new["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
    # ? Create a new df with >78 weeks waiting - prioritise these patients; but if < # of slots, then fill with 18-52 week patients?


# Track used appointment IDs
overbooked_IDs = set()

# Prepare list to store paired appointment IDs
paired_appts = []

# For each very high risk appt, check its duration and pair with the next, longest waiting appt
for overbook_index in range(len(weekswaiting_desc)):
    overbooked_appt = weekswaiting_desc.iloc[overbook_index]
    if (overbooked_appt["appointment_id"] not in overbooked_IDs) and (overbooked_appt["appt_duration"] == dna_very_high_surgeon_new["appt_duration"]):
        paired_appts.append(overbooked_appt)
        # print(overbooked_appt["appt_id"])
        # print(overbooked_appt["weekswaiting"])
        # print(overbooked_appt["duration"])
        # print(overbooked_appt["appt_date"])
        # print(overbooked_appt["appt_time"])
        # print(overbooked_appt["did_not_attend_r

# Pair with the next (longest waiting) available appointment

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().